# Libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)



sys.path.append("../")

from src.utils.impute.pipeline import suburbs, bedrooms, bathrooms, surface, rooms, corrections

n_partitions=16

In [2]:
%load_ext autoreload
%autoreload 2

## Data

In [7]:
df = pd.read_parquet("../data/interim/df_test_eda.parquet", engine="pyarrow")
df = df.head(1500)

In [8]:
df.head(2)

,id,ad_type,lat,lon,country,province,suburb,published_suburb,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,title,description,property_type,operation_type
0,556713,Propiedad,-34.573623,-58.442399,Argentina,Ciudad Autonoma de Buenos Aires,Colegiales,None,3.0,2.0,2.0,NaN,NaN,NaN,USD,departamento venta belgrano capital federal,sup escritura 96.47 m2 cubiertos 61 m2 descubi...,Departamento,Venta
1,192912,Propiedad,-34.606620,-58.430493,Argentina,Ciudad Autonoma de Buenos Aires,Almagro,None,3.0,2.0,2.0,77.0,67.0,NaN,USD,departamento 3 ambientes venta almagro,excelente departamento ambientes amplio modern...,Departamento,Venta


## Impute Missing Values

In [9]:
df.isna().sum()

id                     0
ad_type                0
lat                  109
lon                  109
country                0
province               0
suburb                48
published_suburb    1376
rooms                  0
bedrooms             258
bathrooms             54
surface_total        287
surface_covered      271
price               1500
currency               0
title                  0
description            0
property_type          0
operation_type         0
dtype: int64

### Neighborhood

In [10]:
df = suburbs(df, n_partitions)

Dask Apply:   0%|          | 0/11 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/11 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

### Bedrooms

In [ ]:
df = bedrooms(df, n_partitions)

### Superficies

In [ ]:
df = surface(df, n_partitions)

Pandas Apply:   0%|          | 0/1121 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/1121 [00:00<?, ?it/s]

### Rooms

In [ ]:
df = rooms(df, n_partitions)


### Baño

In [ ]:
df = bathrooms(df, n_partitions)


## Corrections

In [ ]:
df = corrections(df, n_partitions)

# Wrap up

In [ ]:
df.head(2)

,id,ad_type,lat,lon,country,province,suburb,published_suburb,rooms,bedrooms,bathrooms,surface_total,surface_covered,currency,title,description,property_type,operation_type,dist_buenos_aires
0,556713,Propiedad,-34.573623,-58.442399,Argentina,Ciudad Autonoma de Buenos Aires,Colegiales,Colegiales,3.0,2.0,2.0,96.47,2.0,USD,departamento venta belgrano capital federal,sup escritura 96.47 m2 cubiertos 61 m2 descubi...,Departamento,Venta,4.547158
1,192912,Propiedad,-34.606620,-58.430493,Argentina,Ciudad Autonoma de Buenos Aires,Almagro,Almagro,3.0,2.0,2.0,77.00,2.0,USD,departamento 3 ambientes venta almagro,excelente departamento ambientes amplio modern...,Departamento,Venta,1.698298


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 6000 non-null   int64  
 1   ad_type            6000 non-null   object 
 2   lat                6000 non-null   float64
 3   lon                6000 non-null   float64
 4   country            6000 non-null   object 
 5   province           6000 non-null   object 
 6   suburb             6000 non-null   object 
 7   published_suburb   6000 non-null   object 
 8   rooms              6000 non-null   float64
 9   bedrooms           6000 non-null   float64
 10  bathrooms          6000 non-null   float64
 11  surface_total      6000 non-null   float64
 12  surface_covered    6000 non-null   float64
 13  currency           6000 non-null   object 
 14  title              6000 non-null   object 
 15  description        6000 non-null   object 
 16  property_type      6000 

In [ ]:
df.to_parquet("../data/processed/df_test.parquet")

# Wrap Up

In [ ]:
df = df.copy()
# df = df.reset_index(drop=False)

color_scale = [(0, 'orange'), (1,'red')]

fig = None
fig = px.scatter_mapbox(#df[df["dist_buenos_aires"] < 70],
                        df,
                        lat="lat",
                        lon="lon",
                        hover_name="title",
                        hover_data=["id", "suburb", "published_suburb"],
                        color="province",
                        color_continuous_scale=color_scale,
                        zoom=10,
                        height=800,
                        width=1600)

fig.update_layout(mapbox_style="carto-darkmatter")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()